# UMP Infer LSTM (pytorch)

This code is infer LSTM model.

Training code is here:  
https://www.kaggle.com/takamichitoda/ump-train-lstm-pytorch

LSTM model needs to input previous feature’s history.  
That dataset is here:  
https://www.kaggle.com/takamichitoda/ump-features-history

In this code, I infer by concatenating this historical data with the data from the API.

In [ ]:
import numpy as np
import pandas as pd
import random
import os
import gc

import pickle
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from transformers import AdamW
from sklearn.model_selection import KFold

from tqdm.auto import tqdm
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
import ubiquant

device = torch.device("cuda")

In [ ]:
class GCF:
    INPUT_ROOT = "/kaggle/input/ump-npy-dataset/"
    MODEL_ROOT = "/kaggle/input/ump-train-lstm-pytorch"
    SCALER_PATH = "/kaggle/input/ump-npy-dataset/std_scaler.pkl"
    LATEST_FEATURES = '/kaggle/input/ump-features-history/latest_features_128.npy'
    
    SEED = 0
    MAX_LEN = 128
    N_FOLDS = 5
    
    BS = 256
    HIDDEN_SIZE = 128
    FEAT_COLS = [f"f_{i}" for i in range(300)]

In [ ]:
class UMPLSTM(nn.Module):
    def __init__(self):
        super(UMPLSTM, self).__init__()
        conv_out = 64
        pool_k = 16

        self.lstm = nn.LSTM(300, GCF.HIDDEN_SIZE, batch_first=True, num_layers=1, dropout=0.0)        
        self.head = nn.Sequential(
            nn.Linear(GCF.HIDDEN_SIZE, GCF.HIDDEN_SIZE),
            nn.LayerNorm(GCF.HIDDEN_SIZE),
            nn.ReLU(),
            nn.Linear(GCF.HIDDEN_SIZE, 1),
        )
        
        self.criterion = nn.MSELoss(reduction='none')
        
    def forward(self, _x, _y=None):
        out, _ = self.lstm(_x)
        regr = self.head(out)
        regr = regr.squeeze(2)
        
        if _y is None:
            return None, regr
        
        mask = (_y != 999).float()
        loss = self.criterion(regr, _y)
        loss = (loss * mask).mean()
        
        return loss, regr

In [ ]:
load_path = f"{GCF.MODEL_ROOT}/ump_lstm_f0_best_rmse.pth"
model = UMPLSTM()
model.load_state_dict(torch.load(load_path, map_location=torch.device('cpu')))
model.to(device)
model.eval()

In [ ]:
scaler = pickle.load(open(GCF.SCALER_PATH, "rb"))
scaler

In [ ]:
latest_features = np.load(GCF.LATEST_FEATURES)
latest_features.shape

In [ ]:
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    _id = test_df['investment_id'].values
    
    # update new investment_id
    if len(latest_features) <= _id.max():
        add = _id.max() - len(latest_features) + 1
        new_f = np.zeros((add, GCF.MAX_LEN, 300))
        latest_features = np.vstack([latest_features, new_f])
        
    curr_x = scaler.transform(test_df[GCF.FEAT_COLS].values)
    prev_x = torch.tensor(latest_features[_id, 1:])
    x = torch.cat([prev_x, torch.tensor(curr_x).unsqueeze(1)], dim = 1)
    
    with torch.no_grad():
        _, pred = model(x.float().to(device))

    sample_prediction_df['target'] = pred[:, -1].cpu().tolist()  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions
    
    # status update
    latest_features[_id] = x.cpu().numpy()
    
    gc.collect()

In [ ]:
sample_prediction_df